In [1]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import glob
import os
#from keras.applications.vgg16 import preprocess_input
import cv2

In [2]:
#Pre trained MobineNet V2 from TfHub
IMAGE_SHAPE = (224, 224)

classifier_model ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [3]:
#Model Summary
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              3540265   
Total params: 3,540,265
Trainable params: 0
Non-trainable params: 3,540,265
_________________________________________________________________


In [5]:
from scipy import ndimage
def augment(img):
    img1 = np.fliplr(img)
    img2 = np.flipud(img)
    img3 = ndimage.rotate(img,45,reshape=False)
    return img1,img2,img3
def load_image(path):
    #dim =  (299,299)
    img = cv2.imread(path)
    norm_img = np.zeros((256,256))
    img = cv2.normalize(img,  norm_img, 0, 255, cv2.NORM_MINMAX)
    #img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    #processedimage = preprocess_input(img)
    return img

#Load Crack Test Data
def load_data_test(path):
    x = []
    files =glob.glob(path)
    #print(files)
    for file in files:
        x.append(load_image(file))
    x=np.array(x)
    return x


#Load Crack Train Data
def load_data(path):
    x=[]
    y=[]
    c = 0
    dirname, f, filenames= os.walk(path)
    print(dirname)
    cracked = dirname[1][0]
    uncracked = dirname[1][1]
        
    #print(uncracked)
    #print(cracked)
        
    cracked = os.path.join(path,cracked)+"//*"
    uncracked = os.path.join(path,uncracked)+"//*"
    cracked_files = glob.glob(cracked)
    uncracked_files = glob.glob(uncracked)
        
    print(uncracked)
    print(cracked)
    
    for file in cracked_files:
        x.append(load_image(file))
        y.append(1)
            
    
    for file in uncracked_files:
        img = load_image(file)
        img1,img2,img3 = augment(img)
        x.append(img)
        x.append(img1)
        x.append(img2)
        x.append(img3)
        y.append(0)
        y.append(0)
        y.append(0)
        y.append(0)
        
    x = np.array(x)
    y = np.expand_dims(np.array(y),1)
    return x,y

path_train = ".//data//train//"
path_test = ".//data//test//*"

train_x,train_y = load_data(path_train)
test_x = load_data_test(path_test)

print(train_x.shape,train_y.shape,test_x.shape)

('.//data//train//', ['cracked', 'uncracked'], [])
.//data//train//uncracked//*
.//data//train//cracked//*
(37369, 256, 256, 3) (37369, 1) (2000, 256, 256, 3)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
train_x,train_y = shuffle(train_x,train_y, random_state=0)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.10, random_state=42)


In [9]:
#Feature Extractor Headless Model from TFHUB

#feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
#feature_extractor_model = "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4"
#feature_extractor_model = "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/4"
#feature_extractor_layer = hub.KerasLayer(
    #feature_extractor_model, input_shape=(256, 256, 3), trainable=False)

feature_extractor =  tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',input_shape=(256,256,3))
feature_extractor.summary()
#Num of classes to Transfer Learn on
num_classes = 2

#Add Dense last layer for classification for TL
model = tf.keras.Sequential([
  feature_extractor,tf.keras.layers.Dropout(.3),tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [ ]:
#Compiling the New Model
model.compile(
  optimizer=tf.keras.optimizers.Adam(.001),
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['acc'])

#Callbacks
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    self.model.reset_metrics()

batch_stats_callback = CollectBatchStats()

#Train the model
history = model.fit(train_x,tf.keras.utils.to_categorical(
    train_y, num_classes=2, dtype='float32'),validation_data=(val_x,tf.keras.utils.to_categorical(
    val_y, num_classes=2, dtype='float32')),batch_size = 32, epochs=30,
                    callbacks=[batch_stats_callback])

#Plotting Loss based on Callbacks
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats_callback.batch_losses)


Epoch 1/30
1017/1051 [============================>.] - ETA: 4:41 - loss: 0.5648 - acc: 0.7812